**new_test_data : 2020.05.07**
- test_pair 데이터 만들때, 거리기준 0.35 안에 들어야한다!
- bagging 에서는 메모리가 모자르니깐, **만들어진 test_pair 불러온다!**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import train_test_split

import glob
import os
import multiprocessing

import pickle
import time
import random

In [2]:
import seaborn as sns
from sklearn import metrics

---

In [3]:
def chunkIt(seq, num):
    if len(seq) % int(num) == 0:
        avg = int(len(seq) / int(num))
    else:
        avg = int(len(seq) / int(num)) + 1 # 이렇게 해야하네!
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

---

# data load

### - train/test index

In [4]:
os.getcwd()

'/mnt/sda1/myeonggyulee/myeonggyulee/VISION_AI/metric learning/train_within_0_35_balanced'

In [5]:
glob.glob('../2_svm2+data/data/without_validation/*')

['../2_svm2+data/data/without_validation/test_index.pkl',
 '../2_svm2+data/data/without_validation/train_sampling_dict.pkl',
 '../2_svm2+data/data/without_validation/train_pair.pkl',
 '../2_svm2+data/data/without_validation/train_index.pkl',
 '../2_svm2+data/data/without_validation/balanced',
 '../2_svm2+data/data/without_validation/without_balanced']

In [5]:
# load train/text index
with open('../2_svm2+data/data/without_validation/train_index.pkl', 'rb') as f:
    train_index = pickle.load(f)
    
with open('../2_svm2+data/data/without_validation/test_index.pkl', 'rb') as f:
    test_index = pickle.load(f)

---

##  make new-difference vector

### - load train_array

In [7]:
with open('X_new_diff_balanced_train.pkl', 'rb') as f:
    X_diff_train = pickle.load( f )
    
with open('y_new_diff_balanced_train.pkl', 'rb') as f:
    y_diff_train = pickle.load( f )

In [8]:
print( X_diff_train.shape, y_diff_train.shape )

(661038, 256) (661038,)


---

## test set 생성
- signle/bagging 모델 이용시 test_set 활용법 다르다

In [6]:
test_index[:5]

[40648, 66052, 37783, 59761, 16639]

In [7]:
glob.glob('../1_svm/data/*.csv')

['../1_svm/data/new_feat_df.csv']

In [8]:
data = pd.read_csv('../1_svm/data/new_feat_df.csv')

In [9]:
group_no_df = data.groupby('group_no').size().reset_index()
group_no_df.columns = ['group_no', 'size_']

filtering_grp = group_no_df[group_no_df.size_ > 2].group_no.tolist()
data_filter = data[data.group_no.isin( filtering_grp )]

In [10]:
data_filter.shape

(73681, 266)

---

- 거리 데이터

In [11]:
from sklearn.metrics import pairwise_distances

In [12]:
data_renew = data_filter.drop(
            columns = ['title', 'cleaned_title', 'model_id', 'weight', 'top3_word', 'top2_word', 'word_cnt'])

In [13]:
data_renew.iloc[:, 3:].values[:5]

array([[0.09585801, 0.02799749, 0.0276188 , ..., 0.01662012, 0.03039947,
        0.01109296],
       [0.09585799, 0.0279975 , 0.0276188 , ..., 0.01662012, 0.03039948,
        0.01109296],
       [0.09585801, 0.02799749, 0.0276188 , ..., 0.01662012, 0.03039947,
        0.01109296],
       [0.09285977, 0.03225553, 0.02934373, ..., 0.01435256, 0.03604721,
        0.01145816],
       [0.09285977, 0.03225551, 0.02934374, ..., 0.01435256, 0.03604721,
        0.01145816]])

In [14]:
data_renew_distances = pairwise_distances( data_renew.iloc[:, 3:].values, metric='euclidean', n_jobs=-1 )

In [15]:
data_renew_distances.shape

(73681, 73681)

In [16]:
data_renew_distances[:5]

array([[0.00000000e+00, 1.71848574e-07, 0.00000000e+00, ...,
        8.46262232e-01, 8.38199546e-01, 8.15438825e-01],
       [1.71848574e-07, 0.00000000e+00, 1.71848574e-07, ...,
        8.46262223e-01, 8.38199536e-01, 8.15438815e-01],
       [0.00000000e+00, 1.71848574e-07, 0.00000000e+00, ...,
        8.46262232e-01, 8.38199546e-01, 8.15438825e-01],
       [4.00390397e-02, 4.00390180e-02, 4.00390397e-02, ...,
        8.42253810e-01, 8.36874462e-01, 8.12479417e-01],
       [4.00390640e-02, 4.00390424e-02, 4.00390640e-02, ...,
        8.42253781e-01, 8.36874433e-01, 8.12479392e-01]])

----

***Test_pair***
- 거리 기준에 드는 객체만 pair 데이터로 생성하자(1차 0.35)

In [37]:
%%time
# init
test_pair_array = {}
test_label = {}   
start_time = time.time()
# loop
for j,test_ in enumerate(test_index[:100]):
    
    #     init     #
    test_pair_array[test_] = np.array([])
    test_label[test_] = np.array([])
    ################
    
    # pair_array- anchor의 pair 후보 만들기 #
    pair_array = data_filter.iloc[:, 7:-3].values
    label_array = data_filter.group_no.values
    
    #     anchor    #
    anchor_array = pair_array[test_]
    anchor_label = data_filter.iloc[test_].group_no
    
    # pair candidate - 0.35기준 밖 + 자기 자신 index 빼준다 #
    out_index = np.where( data_renew_distances[ test_ ] > 0.35)[0]   # 기준 객체의 pair distance에서 0.35 초과 index
    candidate_array = np.delete( pair_array, [test_] + list(out_index) , axis=0)  # 자기자신 + out_index 지워준다
    candidate_label = np.delete( label_array, [test_] + list(out_index) )
    ################
    
    # append - diff 형태
    test_pair_array[test_] = np.concatenate([np.repeat(anchor_array.reshape(1,-1), len(candidate_array), axis=0 ),
                                             candidate_array], axis=1)
    test_label[test_] = ( candidate_label == anchor_label ).astype(int)
    if (j+1) % 100 == 0:
        elapsed_time = time.time() - start_time
        print(elapsed_time)

8.625832557678223
CPU times: user 4.67 s, sys: 3.95 s, total: 8.63 s
Wall time: 8.63 s


In [40]:
test_

43573

In [39]:
test_pair_array[test_].shape

(3, 512)

***Test_new_diff***
- 거리 기준에 드는 객체만 pair 데이터로 생성하자(1차 0.35)

In [ ]:
%%time
# init
test_pair_array = {}
test_label = {}   
start_time = time.time()
# loop
for j,test_ in enumerate(test_index[:100]):
    
    #     init     #
    test_pair_array[test_] = np.array([])
    test_label[test_] = np.array([])
    ################
    
    # pair_array- anchor의 pair 후보 만들기 #
    pair_array = data_filter.iloc[:, 7:-3].values
    label_array = data_filter.group_no.values
    
    #     anchor    #
    anchor_array = pair_array[test_]
    anchor_label = data_filter.iloc[test_].group_no
    
    # pair candidate - 0.35기준 밖 + 자기 자신 index 빼준다 #
    out_index = np.where( data_renew_distances[ test_ ] > 0.35)[0]   # 기준 객체의 pair distance에서 0.35 초과 index
    candidate_array = np.delete( pair_array, [test_] + list(out_index) , axis=0)  # 자기자신 + out_index 지워준다
    candidate_label = np.delete( label_array, [test_] + list(out_index) )
    ################
    
    # append - diff 형태
    test_pair_array[test_] = (anchor_array - candidate_array)
    test_label[test_] = ( candidate_label == anchor_label ).astype(int)
    if (j+1) % 100 == 0:
        elapsed_time = time.time() - start_time
        print(elapsed_time)

len(test_pair_array)

test_pair_array[40648].shape

---

- save

In [42]:
### pair
with open('../test_data(common)/TEST_within_035/test_pair_v2_035.pkl', 'wb') as f:
    pickle.dump(test_pair_array, f)
    

In [30]:
### diff
with open('../test_data(common)/TEST_within_035/test_pair_035.pkl', 'wb') as f:
    pickle.dump(test_pair_array, f)
    
with open('../test_data(common)/TEST_within_035/test_label_035.pkl', 'wb') as f:
    pickle.dump(test_label, f)

- load

In [9]:
with open('../test_data(common)/TEST_within_035/test_pair_035.pkl', 'rb') as f:
    test_pair_array = pickle.load( f )
    
with open('../test_data(common)/TEST_within_035/test_label_035.pkl', 'rb') as f:
    test_label = pickle.load( f )

---
---

# # Training

In [10]:
import time
import numpy as np
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

# # with rbf
- non-scaling

In [37]:
%%time
n_estimators = 30

estimator = SVC( max_iter = 10000 ) # class_weight : 'balanced'로 해야할까?
bagging = BaggingClassifier(base_estimator = estimator, max_samples=1.0 / n_estimators, 
                        n_estimators=n_estimators, n_jobs=36)
bagging.fit(X_diff_train, y_diff_train)

CPU times: user 1.25 s, sys: 2.12 s, total: 3.37 s
Wall time: 6min 24s


BaggingClassifier(base_estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                     class_weight=None, coef0=0.0,
                                     decision_function_shape='ovr', degree=3,
                                     gamma='scale', kernel='rbf',
                                     max_iter=10000, probability=False,
                                     random_state=None, shrinking=True,
                                     tol=0.001, verbose=False),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=0.03333333333333333, n_estimators=30, n_jobs=36,
                  oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

---

## ## evaluate test set

- prediction 만들기

In [13]:
len(test_label)

100

In [14]:
%%time
prediction_dict = {}

for key_ in list(test_label.keys())[:100]:
    prediction_dict[key_] = bagging.predict(test_pair_array[key_])

CPU times: user 3.9 s, sys: 1 s, total: 4.9 s
Wall time: 4.71 s


In [15]:
len(prediction_dict)

100

In [16]:
from sklearn import metrics

- test_set에 대한 성능 살피기
> precision / recall

## # rbf(=default)

### 1. max_iter 100 & n_estimators 30

In [17]:
precision_100 = []
recall_100 = []

for ind, predicted in prediction_dict.items(): 
    print(ind)
    print('label_size : {}'.format(np.sum(test_label[ind])), 
          'prediction_size : {}'.format(np.sum(predicted)) )
    
    # calculate precision / recall
    precision_ = metrics.precision_score( test_label[ind] , predicted )
    recall_ = metrics.recall_score( test_label[ind] , predicted )
    
    # print
    print( precision_ )
    print( recall_ )
    print('\n')
    
    # append precision / recall
    precision_100.append( precision_ )
    recall_100.append( recall_ )
    

40648
label_size : 2 prediction_size : 1.0
1.0
0.5


66052
label_size : 2 prediction_size : 17.0
0.11764705882352941
1.0


37783
label_size : 2 prediction_size : 0.0
0.0
0.0


59761
label_size : 2 prediction_size : 2.0
1.0
1.0


16639
label_size : 4 prediction_size : 4.0
1.0
1.0


17958
label_size : 5 prediction_size : 5.0
1.0
1.0


10918
label_size : 5 prediction_size : 18.0
0.2777777777777778
1.0


26542
label_size : 4 prediction_size : 4.0
1.0
1.0


71651
label_size : 2 prediction_size : 2.0
1.0
1.0


45696
label_size : 1 prediction_size : 1.0
1.0
1.0


4253
label_size : 8 prediction_size : 12.0
0.6666666666666666
1.0


63461
label_size : 2 prediction_size : 25.0
0.08
1.0


572
label_size : 14 prediction_size : 17.0
0.8235294117647058
1.0


66460
label_size : 0 prediction_size : 0.0
0.0
0.0


70828
label_size : 1 prediction_size : 0.0
0.0
0.0


34741
label_size : 4 prediction_size : 3.0
1.0
0.75


14588
label_size : 5 prediction_size : 0.0
0.0
0.0


8647
label_size : 4 prediction_si

/home/wmind/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wmind/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.42857142857142855
1.0


14092
label_size : 6 prediction_size : 6.0
1.0
1.0


29533
label_size : 4 prediction_size : 4.0
1.0
1.0


72633
label_size : 2 prediction_size : 2.0
1.0
1.0


19729
label_size : 4 prediction_size : 10.0
0.4
1.0


63959
label_size : 2 prediction_size : 46.0
0.043478260869565216
1.0


65697
label_size : 1 prediction_size : 1.0
1.0
1.0


63092
label_size : 2 prediction_size : 2.0
1.0
1.0


12067
label_size : 1 prediction_size : 1.0
1.0
1.0


9712
label_size : 6 prediction_size : 10.0
0.6
1.0


35832
label_size : 3 prediction_size : 7.0
0.42857142857142855
1.0


50058
label_size : 3 prediction_size : 3.0
1.0
1.0


8878
label_size : 1 prediction_size : 0.0
0.0
0.0


10198
label_size : 8 prediction_size : 12.0
0.6666666666666666
1.0


7132
label_size : 7 prediction_size : 7.0
1.0
1.0


19158
label_size : 6 prediction_size : 2.0
1.0
0.3333333333333333


71760
label_size : 2 prediction_size : 2.0
1.0
1.0


29882
label_size : 3 prediction_size : 3.0
1.0
1.0


2487
labe

In [18]:
print(len(precision_100), len(recall_100) )

100 100


In [19]:
np.mean(precision_100)

0.6251209638888005

In [20]:
np.mean(recall_100)

0.7579166666666667

---
---

# # with poly3

In [30]:
%%time
n_estimators = 30

estimator = SVC( kernel='poly', max_iter = 1000 ) # class_weight : 'balanced'로 해야할까?
bagging_poly = BaggingClassifier(base_estimator = estimator, max_samples=1.0 / n_estimators, 
                        n_estimators=n_estimators, n_jobs=36)
bagging_poly.fit(X_diff_train, y_diff_train)

CPU times: user 585 ms, sys: 1.7 s, total: 2.29 s
Wall time: 40.5 s


BaggingClassifier(base_estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                     class_weight=None, coef0=0.0,
                                     decision_function_shape='ovr', degree=3,
                                     gamma='scale', kernel='poly',
                                     max_iter=1000, probability=False,
                                     random_state=None, shrinking=True,
                                     tol=0.001, verbose=False),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=0.03333333333333333, n_estimators=30, n_jobs=36,
                  oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

# # bagging_poly3 결과

- prediction 만들기

In [31]:
%%time
prediction_poly3_dict = {}

for key_ in list(test_label.keys())[:100]:
    prediction_poly3_dict[key_] = bagging_poly.predict(test_pair_array[key_])

CPU times: user 5.68 s, sys: 3.27 s, total: 8.95 s
Wall time: 8.52 s


In [32]:
len(prediction_poly3_dict)

100

### 테스트 결과 (test_index[:100])

In [33]:
precision_100 = []
recall_100 = []

for ind, predicted in prediction_poly3_dict.items(): 
    print(ind)
    print('label_size : {}'.format(np.sum(test_label[ind])), 
          'prediction_size : {}'.format(np.sum(predicted)) )
    
    # calculate precision / recall
    precision_ = metrics.precision_score( test_label[ind] , predicted )
    recall_ = metrics.recall_score( test_label[ind] , predicted )
    
    # print
    print( precision_ )
    print( recall_ )
    print('\n')
    
    # append precision / recall
    precision_100.append( precision_ )
    recall_100.append( recall_ )
    

40648
label_size : 2 prediction_size : 10.0
0.2
1.0


66052
label_size : 2 prediction_size : 17.0
0.11764705882352941
1.0


37783
label_size : 2 prediction_size : 5.0
0.4
1.0


59761
label_size : 2 prediction_size : 5.0
0.4
1.0


16639
label_size : 4 prediction_size : 4.0
1.0
1.0


17958
label_size : 5 prediction_size : 5.0
1.0
1.0


10918
label_size : 5 prediction_size : 18.0
0.2777777777777778
1.0


26542
label_size : 4 prediction_size : 4.0
1.0
1.0


71651
label_size : 2 prediction_size : 5.0
0.4
1.0


45696
label_size : 1 prediction_size : 2.0
0.5
1.0


4253
label_size : 8 prediction_size : 12.0
0.6666666666666666
1.0


63461
label_size : 2 prediction_size : 96.0
0.020833333333333332
1.0


572
label_size : 14 prediction_size : 17.0
0.8235294117647058
1.0


66460
label_size : 0 prediction_size : 1.0
0.0
0.0


70828
label_size : 1 prediction_size : 24.0
0.041666666666666664
1.0


34741
label_size : 4 prediction_size : 170.0
0.023529411764705882
1.0


14588
label_size : 5 prediction_s

/home/wmind/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1.0
1.0


19158
label_size : 6 prediction_size : 9.0
0.6666666666666666
1.0


71760
label_size : 2 prediction_size : 2.0
1.0
1.0


29882
label_size : 3 prediction_size : 7.0
0.42857142857142855
1.0


2487
label_size : 12 prediction_size : 25.0
0.48
1.0


45553
label_size : 2 prediction_size : 11.0
0.18181818181818182
1.0


41970
label_size : 2 prediction_size : 18.0
0.1111111111111111
1.0


37904
label_size : 4 prediction_size : 64.0
0.0625
1.0


23021
label_size : 4 prediction_size : 93.0
0.043010752688172046
1.0


16687
label_size : 5 prediction_size : 5.0
1.0
1.0


20559
label_size : 3 prediction_size : 322.0
0.009316770186335404
1.0


71620
label_size : 1 prediction_size : 8.0
0.125
1.0


35756
label_size : 3 prediction_size : 7.0
0.42857142857142855
1.0


65107
label_size : 0 prediction_size : 1.0
0.0
0.0


62058
label_size : 1 prediction_size : 7.0
0.14285714285714285
1.0


68572
label_size : 2 prediction_size : 29.0
0.06896551724137931
1.0


30575
label_size : 5 prediction_size 

In [34]:
print(len(precision_100), len(recall_100) )

100 100


In [35]:
np.mean(precision_100)

0.5438155954453614

In [36]:
np.mean(recall_100)

0.96